Version 1.1.0

# Mean encodings

In this programming assignment you will be working with `1C` dataset from the final competition. You are asked to encode `item_id` in 4 different ways:

    1) Via KFold scheme;  
    2) Via Leave-one-out scheme;
    3) Via smoothing scheme;
    4) Via expanding mean scheme.

**You will need to submit** the correlation coefficient between resulting encoding and target variable up to 4 decimal places.

### General tips

* Fill NANs in the encoding with `0.3343`.
* Some encoding schemes depend on sorting order, so in order to avoid confusion, please use the following code snippet to construct the data frame. This snippet also implements mean encoding without regularization.

In [19]:
import pandas as pd
import numpy as np
from itertools import product
from grader import Grader

# Read data

In [20]:
sales = pd.read_csv('../readonly/final_project_data/sales_train.csv.gz')

# Aggregate data

Since the competition task is to make a monthly prediction, we need to aggregate the data to montly level before doing any encodings. The following code-cell serves just that purpose.

In [21]:
index_cols = ['shop_id', 'item_id', 'date_block_num']

# For every month we create a grid from all shops/items combinations from that month
grid = [] 
for block_num in sales['date_block_num'].unique():
    cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
    cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
    grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

#turn the grid into pandas dataframe
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

#get aggregated values for (shop_id, item_id, month)
gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'}})

#fix column names
gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
#join aggregated data to the grid
all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
#sort the data
all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)

/opt/conda/lib/python3.6/site-packages/pandas/core/groupby.py:4036: FutureWarning: using a dict with renaming is deprecated and will be removed in a future version
  return super(DataFrameGroupBy, self).aggregate(arg, *args, **kwargs)


# Mean encodings without regularization

After we did the techinical work, we are ready to actually *mean encode* the desired `item_id` variable. 

Here are two ways to implement mean encoding features *without* any regularization. You can use this code as a starting point to implement regularized techniques. 

#### Method 1

In [22]:
# Calculate a mapping: {item_id: target_mean}
item_id_target_mean = all_data.groupby('item_id').target.mean()

# In our non-regularized case we just *map* the computed means to the `item_id`'s
all_data['item_target_enc'] = all_data['item_id'].map(item_id_target_mean)

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.483038698862


#### Method 2

In [23]:
'''
    Differently to `.target.mean()` function `transform` 
   will return a dataframe with an index like in `all_data`.
   Basically this single line of code is equivalent to the first two lines from of Method 1.
'''
all_data['item_target_enc'] = all_data.groupby('item_id')['target'].transform('mean')

# Fill NaNs
all_data['item_target_enc'].fillna(0.3343, inplace=True) 

# Print correlation
encoded_feature = all_data['item_target_enc'].values
print(np.corrcoef(all_data['target'].values, encoded_feature)[0][1])

0.483038698862


See the printed value? It is the correlation coefficient between the target variable and your new encoded feature. You need to **compute correlation coefficient** between the encodings, that you will implement and **submit those to coursera**.

In [24]:
grader = Grader()

# 1. KFold scheme

Explained starting at 41 sec of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

**Now it's your turn to write the code!** 

You may use 'Regularization' video as a reference for all further tasks.

First, implement KFold scheme with five folds. Use KFold(5) from sklearn.model_selection. 

1. Split your data in 5 folds with `sklearn.model_selection.KFold` with `shuffle=False` argument.
2. Iterate through folds: use all but the current fold to calculate mean target for each level `item_id`, and  fill the current fold.

    *  See the **Method 1** from the example implementation. In particular learn what `map` and pd.Series.map functions do. They are pretty handy in many situations.

In [25]:
all_data.head()

,shop_id,item_id,date_block_num,target,item_target_enc
139255,0,19,0,0.0,0.022222
141495,0,27,0,0.0,0.056834
144968,0,28,0,0.0,0.141176
142661,0,29,0,0.0,0.037383
138947,0,32,0,6.0,1.319042


In [26]:
from sklearn.model_selection import KFold

kf = KFold(n_splits = 5, shuffle = False, random_state = 123)

for tr_idx, val_idx in kf.split(all_data):
    X_tr, X_val = all_data.iloc[tr_idx], all_data.iloc[val_idx]
    
    # mean encoding of validation data depends on the training data
    X_tr_group = X_tr.groupby('item_id')['target']
    X_val['item_target_enc'] = X_val['item_id'].map(X_tr_group.mean())
    
    # copy the mean encoding back to the entire dataset
    all_data.iloc[val_idx] = X_val
    
prior = all_data['target'].mean()
all_data['item_target_enc'].fillna(prior, inplace=True)
encoded_feature = all_data['item_target_enc'].values

# You will need to compute correlation like that
corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('KFold_scheme', corr)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


0.416459048853
Current answer for task KFold_scheme is: 0.416459048853


In [27]:
all_data.head()

,shop_id,item_id,date_block_num,target,item_target_enc
139255,0,19,0,0.0,0.334273
141495,0,27,0,0.0,0.048523
144968,0,28,0,0.0,0.142424
142661,0,29,0,0.0,0.030303
138947,0,32,0,6.0,0.894020


In [28]:
target = all_data['target'].values  # target variable
kf = KFold(n_splits = 5, shuffle = False, random_state = 123)

# use this approach for multiple columns
cols = ['item_id']

# outer loop iterates through the K-Fold iterator to separate the training data into chunks. X_tr is used to estimate
#  the encoding and X_val is used to apply the encoding
for tr_idx, val_idx in kf.split(target):
    X_tr, X_val = all_data.iloc[tr_idx], all_data.iloc[val_idx]
    # inner loop iterates through all the columns we wish to encode to map the estimated encodings to the X_val
    #  dataframe
    for col in cols:  # iterate through the columns we wish to encode
        means = X_val[col].map(X_tr.groupby(col).target.mean())
        X_val[col+'_mean_target'] = means
    # here, we fill the new training data frame with the result
    all_data.iloc[val_idx] = X_val
    
prior = all_data['target'].mean()  # global mean
all_data.fillna(prior, inplace = True)  # fill NANs with the global mean. These occur because rare categories may only
                                         #  be present in a single fold. So there is no data to estimate the target
                                         #  mean for them. And hence NANs appear.

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [29]:
all_data.head()

,shop_id,item_id,date_block_num,target,item_target_enc
139255,0,19,0,0.0,0.334273
141495,0,27,0,0.0,0.048523
144968,0,28,0,0.0,0.142424
142661,0,29,0,0.0,0.030303
138947,0,32,0,6.0,0.894020


# 2. Leave-one-out scheme

Now, implement leave-one-out scheme. Note that if you just simply set the number of folds to the number of samples and run the code from the **KFold scheme**, you will probably wait for a very long time. 

To implement a faster version, note, that to calculate mean target value using all the objects but one *given object*, you can:

1. Calculate sum of the target values using all the objects.
2. Then subtract the target of the *given object* and divide the resulting value by `n_objects - 1`. 

Note that you do not need to perform `1.` for every object. And `2.` can be implemented without any `for` loop.

It is the most convenient to use `.transform` function as in **Method 2**.

In [30]:
# calculate sum of target values using all the objects
target_sum = all_data.groupby('item_id')['target'].transform('sum')

# subtract the target of the given object ... and divide by the number of objects minus 1
n_objects = all_data.groupby('item_id')['target'].transform('count')
all_data['item_target_enc'] = ( target_sum - all_data['target'] ) / ( n_objects - 1)

encoded_feature = all_data['item_target_enc'].values

corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Leave-one-out_scheme', corr)

0.480384831129
Current answer for task Leave-one-out_scheme is: 0.480384831129


In [31]:
all_data.head()

,shop_id,item_id,date_block_num,target,item_target_enc
139255,0,19,0,0.0,0.022727
141495,0,27,0,0.0,0.056911
144968,0,28,0,0.0,0.141414
142661,0,29,0,0.0,0.037500
138947,0,32,0,6.0,1.316088


# 3. Smoothing

Explained starting at 4:03 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

Next, implement smoothing scheme with $\alpha = 100$. Use the formula from the first slide in the video and $0.3343$ as `globalmean`. Note that `nrows` is the number of objects that belong to a certain category (not the number of rows in the dataset).

In [32]:
# establish the equation parameters
target_mean = all_data.groupby('item_id')['target'].transform('mean')
global_mean = all_data['target'].mean()
nrows = all_data.groupby('item_id')['target'].transform('count')
alpha = 100

# calculate the values and fill missing values with NANs
all_data['item_target_enc'] = ( target_mean * nrows + global_mean * alpha ) / ( nrows + alpha )
all_data.fillna(global_mean, inplace = True)

encoded_feature = all_data['item_target_enc'].values

corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Smoothing_scheme', corr)

0.481819889227
Current answer for task Smoothing_scheme is: 0.481819889227


In [33]:
all_data.head()

,shop_id,item_id,date_block_num,target,item_target_enc
139255,0,19,0,0.0,0.237430
141495,0,27,0,0.0,0.089901
144968,0,28,0,0.0,0.168960
142661,0,29,0,0.0,0.107903
138947,0,32,0,6.0,1.260633


# 4. Expanding mean scheme

Explained starting at 5:50 of [Regularization video](https://www.coursera.org/learn/competitive-data-science/lecture/LGYQ2/regularization).

Finally, implement the *expanding mean* scheme. It is basically already implemented for you in the video, but you can challenge yourself and try to implement it yourself. You will need [`cumsum`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.cumsum.html) and [`cumcount`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.GroupBy.cumcount.html) functions from pandas.

In [37]:
# cumsum stores the cumulative sum of the target variable up to the given row
# cumcount stores the cumulative count
col = 'item_id'
cumsum = all_data.groupby(col)['target'].cumsum() - all_data['target']
cumcnt = all_data.groupby(col).cumcount()

all_data[col+' mean target'] = cumsum / cumcnt
all_data.fillna(global_mean, inplace = True)

encoded_feature = all_data['item_id mean target'].values

corr = np.corrcoef(all_data['target'].values, encoded_feature)[0][1]
print(corr)
grader.submit_tag('Expanding_mean_scheme', corr)

0.502524518991
Current answer for task Expanding_mean_scheme is: 0.502524518991


In [36]:
all_data.head()

,shop_id,item_id,date_block_num,target,item_target_enc,item_id mean target
139255,0,19,0,0.0,0.237430,NaN
141495,0,27,0,0.0,0.089901,NaN
144968,0,28,0,0.0,0.168960,NaN
142661,0,29,0,0.0,0.107903,NaN
138947,0,32,0,6.0,1.260633,NaN


## Authorization & Submission
To submit assignment parts to Cousera platform, please, enter your e-mail and token into variables below. You can generate token on this programming assignment page. Note: Token expires 30 minutes after generation.

In [38]:
STUDENT_EMAIL = 'nikkifitzherbert@gmail.com'
STUDENT_TOKEN = 'ZJe3siiJv03HjgAg'
grader.status()

You want to submit these numbers:
Task KFold_scheme: 0.416459048853
Task Leave-one-out_scheme: 0.480384831129
Task Smoothing_scheme: 0.481819889227
Task Expanding_mean_scheme: 0.502524518991


In [39]:
grader.submit(STUDENT_EMAIL, STUDENT_TOKEN)

Submitted to Coursera platform. See results on assignment page!
